In [1]:
#criar um navegador

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

servico = Service(ChromeDriverManager().install())

navegador = webdriver.Chrome(service=servico)
navegador.maximize_window()


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Gustavo Dias Martins\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [2]:
#importar/vizualizar a base de dados

import pandas as pd

tabela_produtos = pd.read_excel(r'buscas.xlsx')
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,Mini Watch,3000,3500
1,rtx 3060,Zota Galax,4000,4500


# Definição das funções

In [3]:
def busca_google_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo):

    #entrar no google
    navegador.get('https://www.google.com/')
    
    #tratar os valores que vieram da tabela
    produto = produto.lower()
    lista_termos_produto = produto.split(' ')
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)


    #procurar o produto e dar Enter
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(produto, Keys.ENTER)

    #clicar na opção de Shopping
    navegador.find_element(By.LINK_TEXT, 'Shopping').click()

    #percorrer esses resultados pegando o preço, nome, site caso estejam dentro das condições
    lista_ofertas = [] #lista que a função vai dar resposta
    
    #enquanto tiver o dotão de próxima pagina
    while len(navegador.find_elements(By.ID, 'pnnext')) > 0:
        #criar uma lista com todos resultados
        lista_resultados = navegador.find_elements(By.CLASS_NAME, 'i0X6df')

        for resultado in lista_resultados:
            nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
            nome = nome.lower()

            #verificação do nome, se tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True

            #verificação se no nome tem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False

            #se tem_termos_banidos = False e tem_todos_termos_produto = True
            if not tem_termos_banidos and tem_todos_termos_produto:

                #tratar o preço
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                if '+' in preco:
                    continue
                else:
                    preco = preco.replace('R$','').replace(' ','').replace('.','').replace(',','.')
                    preco = float(preco)

                #verificar se o preço está entre o minimo e maximo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'vhAUVb')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome,preco,link))
        navegador.find_element(By.ID, 'pnnext').click()
    return lista_ofertas

In [4]:
import time

def busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo):

    navegador.get('https://www.buscape.com.br/')

    #tratar os valores que vieram da tabela
    produto = produto.lower()
    lista_termos_produto = produto.split(' ')
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)


    navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__FInnF').send_keys(produto, Keys.ENTER)

    time.sleep(5)

    lista_ofertas = []

    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')

    for resultado in lista_resultados:
        nome = resultado.get_attribute('title')
        nome = nome.lower()
        preco = resultado.find_element(By.CLASS_NAME,'CellPrice_MainValue__JXsj_').text
        link = resultado.get_attribute('href')

        #verificação do nome, se tem algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

            #verificação se no nome tem todos os termos do nome do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

            #se tem_termos_banidos = False e tem_todos_termos_produto = True
        if not tem_termos_banidos and tem_todos_termos_produto:

            #tratar o preço
            preco = preco.replace('R$','').replace(' ','').replace('.','').replace(',','.')
            preco = float(preco)

            #verificar se o preço está entre o minimo e maximo
            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas.append((nome,preco,link))
         
    return lista_ofertas

# Construção da base de ofertas

In [5]:
tabela_ofertas = pd.DataFrame()
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha,'Nome']
    termos_banidos = tabela_produtos.loc[linha,'Termos banidos']
    preco_minimo = tabela_produtos.loc[linha,'Preço mínimo']
    preco_maximo = tabela_produtos.loc[linha, 'Preço máximo']
    lista_ofertas_google_shopping = busca_google_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    
    #se a lista não estiver vazia
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['nome', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    
    #se a lista não estiver vazia
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['nome', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscapea = None
        
display(tabela_ofertas)

,nome,preco,link
0,"iphone 11 preto, com tela de 6,1, 4g, 64 gb e ...",3499.00,https://www.google.com/url?url=https://www.nov...
1,"iphone 11 pro, com tela de 5,8"", 4g, 64 gb e c...",3298.97,https://www.google.com/url?url=https://www.out...
2,"iphone 11 apple 64gb amarelo 6,1 12mp - ios",3437.48,https://www.google.com/url?url=https://www.ame...
3,"iphone 11 apple 64gb branco, tela de 6,1”, câm...",3499.90,https://www.google.com/url?url=https://www.for...
4,"iphone 11, preto, 64gb, tela hd 6,1, câm. 12mp...",3099.00,https://www.google.com/url?url=https://lojaonl...
5,iphone xr apple 64gb câmera 12mp tela liquid r...,3069.00,https://www.google.com/url?url=https://www.kab...
6,"iphone se apple tela retina hd de 4.7”, ios, 6...",3099.00,https://www.google.com/url?url=http://www.sipo...
0,smartphone apple iphone se 2 vermelho 64gb 12....,3002.00,https://www.buscape.com.br/celular/smartphone-...
0,placa de vídeo evga geforce rtx 3060 xc gaming...,4130.90,https://www.google.com/url?url=https://www.cas...
1,placa de vídeo asus rtx 3060 dual lhr v2 12gb ...,4190.00,https://www.google.com/url?url=https://www.ext...


# Exportar para excel

In [6]:
tabela_ofertas.to_excel('Ofertas.xlsx',index= False)

# Enviar email

In [7]:
import smtplib
import email.message
import mimetypes
import os
from email.message import EmailMessage

def enviar_email():  
    corpo_email = f"""
    <p>Bom Dia,</p>
    <p></p>
    <p>Segue em anexo a tabela com os produtos encontrados.</p>
    <p>{tabela_ofertas.to_html(index= False)}</p>
   <p>Att, Gustavo</p>
    """

    msg = EmailMessage()
    msg['Subject'] = "Busca Produtos"
    msg['From'] = 'XXXXX@gmail.com'
    msg['To'] = 'XXXXX@gmail.com'
    password = 'jsjdgcowqpacsad' 
    msg.add_header('Content-Type', 'text/html')
    msg.set_content(corpo_email, subtype='html')
    

    #Criar o anexo
    caminho_arquivo = r'C:\Users\Gustavo\Documents\Python\25-Projetos\Projeto WebScrapping\Ofertas.xlsx'
    
    mime_type, encoding = mimetypes.guess_type(caminho_arquivo)
    with open(caminho_arquivo, 'rb') as fp:
        dados = fp.read()
    msg.add_attachment(dados, maintype=mime_type.split("/")[0], subtype=mime_type.split("/")[1], filename=f"{os.path.basename(caminho_arquivo)}")
    
    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    # Login Credentials for sending the mail
    s.login(msg['From'], password)
    s.send_message(msg)
    print('Email enviado')


In [8]:
if len(tabela_ofertas) >0:
    enviar_email()
navegador.quit()

Email enviado
